In [1]:
# | default_exp blocks/heads_3d

# Imports

In [2]:
# | export

from torch import nn
from segmentation_models_pytorch.base.modules import Activation

In [3]:
import torch

# Main classes

In [4]:
# | export

# Inspiration: https://github.com/qubvel-org/segmentation_models.pytorch/blob/main/segmentation_models_pytorch/base/heads.py
class ClassificationHead3D(nn.Sequential):
    def __init__(self, in_channels, classes, pooling="avg", dropout=0.2, activation=None):
        if pooling not in ("max", "avg"):
            raise ValueError("Pooling should be one of ('max', 'avg'), got {}.".format(pooling))
        pool = nn.AdaptiveAvgPool3d(1) if pooling == "avg" else nn.AdaptiveMaxPool3d(1)
        flatten = nn.Flatten()
        dropout = nn.Dropout(p=dropout, inplace=True) if dropout else nn.Identity()
        linear = nn.Linear(in_channels, classes, bias=True)
        activation = Activation(activation)
        super().__init__(pool, flatten, dropout, linear, activation)

In [5]:
test_model = ClassificationHead3D(32, 3)
test_input = torch.randn(5, 32, 8, 8, 8)
test_output = test_model(test_input)
test_output.shape

torch.Size([5, 3])

# nbdev

In [6]:
!nbdev_export